In [1]:
# --- Mount Google Drive ---
from google.colab import drive
drive.mount('/content/drive')

# import os

# # --- Create output folder inside Drive ---
# save_dir = '/content/drive/MyDrive/bengali_corpus'
# os.makedirs(save_dir, exist_ok=True)


Mounted at /content/drive


In [2]:

# # --- File path in Drive ---
# output_file = os.path.join(save_dir, 'bengali_corpus.txt')

# # --- Load Bengali Wikipedia dataset ---
# from datasets import load_dataset
# from tqdm import tqdm
# import re
# import os

# wiki = load_dataset("wikimedia/wikipedia", "20231101.bn", split="train")

# min_length = 100  # skip very short articles

# with open(output_file, 'w', encoding='utf-8') as f:
#     for article in tqdm(wiki, desc="Saving articles"):
#         text = article['text'].strip()
#         if len(text) < min_length:
#             continue

#         # basic cleaning
#         text = re.sub(r'\[\d+\]', '', text)   # remove [1], [2], etc.
#         text = re.sub(r'\n+', '\n', text)     # collapse newlines

#         f.write(text + '\n\n')

# # --- Show stats ---
# size_mb = os.path.getsize(output_file) / (1024**2)
# print(f"Saved corpus to: {output_file}")
# print(f"Corpus size: {size_mb:.2f} MB")

In [3]:
corpus_path = '/content/drive/MyDrive/bengali_corpus/bengali_corpus.txt'

with open(corpus_path, 'r', encoding='utf-8') as f:
    for _ in range(10):  # print first 10 lines
        print(f.readline().strip())

বাংলা ভাষা (বাঙলা, বাঙ্গলা, তথা বাঙ্গালা নামেও পরিচিত) একটি ইন্দো-আর্য ভাষা, যা দক্ষিণ এশিয়ার বাঙালি জাতির প্রধান কথ্য ও লেখ্য ভাষা। মাতৃভাষীর সংখ্যায় বাংলা ইন্দো-ইউরোপীয় ভাষা পরিবারের পঞ্চম ও মোট ব্যবহারকারীর সংখ্যা অনুসারে বাংলা বিশ্বের ষষ্ঠ বৃহত্তম ভাষা। বাংলা সার্বভৌম ভাষাভিত্তিক জাতিরাষ্ট্র বাংলাদেশের একমাত্র রাষ্ট্রভাষা তথা সরকারি ভাষা এবং ভারতের পশ্চিমবঙ্গ, ত্রিপুরা, আসামের বরাক উপত্যকার সরকারি ভাষা। বঙ্গোপসাগরে অবস্থিত আন্দামান দ্বীপপুঞ্জের প্রধান কথ্য ভাষা বাংলা। এছাড়া ভারতের ঝাড়খণ্ড, বিহার, মেঘালয়, মিজোরাম,  ওড়িশা রাজ্যগুলোতে উল্লেখযোগ্য পরিমাণে বাংলাভাষী জনগণ রয়েছে। ২০১১ সালের আদমশুমারি অনুযায়ী, ভারতের মোট জনসংখ্যার ৮.০৩ শতাংশ মানুষ বাংলা ভাষায় কথা বলে এবং হিন্দির পরেই ভারতে সর্বাধিক প্রচলিত ভাষা - বাংলা। এছাড়াও মধ্য প্রাচ্য, আমেরিকা ও ইউরোপে উল্লেখযোগ্য পরিমাণে বাংলাভাষী অভিবাসী রয়েছে। সারা বিশ্বে সব মিলিয়ে ২৭.৬ কোটির অধিক লোক দৈনন্দিন জীবনে বাংলা ব্যবহার করে। বাংলাদেশের জাতীয় সঙ্গীত এবং ভারতের জাতীয় সঙ্গীত ও স্তোত্র বাংলাতে রচিত।
বাংলা ভাষা বিকাশের ইতিহাস ১৩

In [4]:
# # with open('bengali_corpus.txt', 'r', encoding='utf-8') as f:
# #     articles = f.read().split('\n\n')  # each article is one element

# # print(f"Total articles: {len(articles)}")
# # print("\n--- First article ---\n")
# # print(articles[0])
# num_chars = sum(len(a) for a in articles)
# num_tokens_approx = sum(len(a.split()) for a in articles)  # rough word count

# print(f"Total characters: {num_chars}")
# print(f"Approx. total tokens: {num_tokens_approx}")

In [5]:
with open(corpus_path, "r", encoding="utf-8") as f:
    text = f.read()

In [6]:
# Byte-level encoding
tokens = list(text.encode("utf-8"))  # each byte 0–255
print("Initial tokens:", len(tokens))

Initial tokens: 926042697


In [2]:
import math
import os
import json
from tqdm import tqdm

# --- 1️⃣ Load only 5 MB of data ---
corpus_path = '/content/drive/MyDrive/bengali_corpus/bengali_corpus.txt'

with open(corpus_path, "r", encoding="utf-8") as f:
    text = f.read()

sample_bytes = text.encode("utf-8")[:5 * 1024 * 1024]
tokens = list(sample_bytes)
print(f"Training on {len(tokens):,} byte tokens (~5 MB of data)")

# --- 2️⃣ Define helper functions ---
def get_stats(ids):
    counts = {}
    for pair in zip(ids, ids[1:]):
        counts[pair] = counts.get(pair, 0) + 1
    return counts

def merge(ids, pair, idx):
    newids = []
    i = 0
    while i < len(ids):
        if i < len(ids) - 1 and ids[i] == pair[0] and ids[i + 1] == pair[1]:
            newids.append(idx)
            i += 2
        else:
            newids.append(ids[i])
            i += 1
    return newids

# --- 3️⃣ Train BPE (recompute every merge) ---
vocab_size = 500           # target vocab size
num_merges = vocab_size - 256
ids = list(tokens)

print(f"\n🚀 Starting BPE training: {num_merges} merges (full recomputation each step)...\n")

merges = []  # track all merges in order (each is a tuple of ints)
with tqdm(total=num_merges, desc="BPE Training Progress", ncols=100) as pbar:
    for merge_counter in range(num_merges):
        stats = get_stats(ids)
        if not stats:
            print("⚠️ No more valid pairs to merge — stopping early.")
            break

        pair = max(stats, key=stats.get)
        idx = 256 + merge_counter
        ids = merge(ids, pair, idx)
        merges.append(pair)
        pbar.update(1)

        if (merge_counter + 1) % 100 == 0 or merge_counter == num_merges - 1:
            compression_ratio = len(tokens) / len(ids)
            print(f"📦 Merges: {merge_counter+1}/{num_merges} "
                  f"| Current compression: {compression_ratio:.2f}× "
                  f"| Current length: {len(ids):,}")

# --- 4️⃣ Final stats ---
final_ratio = len(tokens) / len(ids)
print("\n✅ Training complete!")
print(f"✅ Final vocabulary size: {vocab_size}")
print(f"✅ Final compression ratio: {final_ratio:.2f}×")

# --- 5️⃣ Save artifacts (merges.txt, vocab.json, config.json) ---
save_dir = '/content/drive/MyDrive/bengali_tokenizer'
os.makedirs(save_dir, exist_ok=True)

# 5.1 merges.txt
merges_path = os.path.join(save_dir, 'merges.txt')
with open(merges_path, 'w', encoding='utf-8') as f:
    for a, b in merges:
        f.write(f"{a} {b}\n")

# 5.2 vocab.json – include readable mappings for all merged tokens
vocab = {}

# Base byte tokens (0–255)
for i in range(256):
    vocab[str(i)] = chr(i)

# Add merged tokens (≥256)
for idx, (a, b) in enumerate(merges, start=256):
    a_char = vocab.get(str(a), f"<{a}>")
    b_char = vocab.get(str(b), f"<{b}>")
    vocab[str(idx)] = a_char + b_char

vocab["_note"] = (
    "Base byte tokens 0–255 stored above; merged tokens 256+ constructed "
    "recursively from their parent pairs."
)

vocab_path = os.path.join(save_dir, 'vocab.json')
with open(vocab_path, 'w', encoding='utf-8') as f:
    json.dump(vocab, f, ensure_ascii=False, indent=2)

# 5.3 config.json
config = {
    "vocab_size": vocab_size,
    "num_merges_saved": len(merges),
    "compression_ratio": round(final_ratio, 4),
    "merges_start_index": 256,
    "notes": "Byte-level BPE tokenizer for Bengali corpus; includes mappings for merged tokens >255."
}
config_path = os.path.join(save_dir, 'config.json')
with open(config_path, 'w', encoding='utf-8') as f:
    json.dump(config, f, indent=2)

print(f"\n📁 Saved -> {merges_path}")
print(f"📁 Saved -> {vocab_path}")
print(f"📁 Saved -> {config_path}")
print(f"✅ All tokenizer artifacts saved in: {save_dir}")


Training on 5,242,880 byte tokens (~5 MB of data)

🚀 Starting BPE training: 244 merges (full recomputation each step)...



BPE Training Progress:  41%|███████████████▌                      | 100/244 [01:43<02:26,  1.02s/it]

📦 Merges: 100/244 | Current compression: 3.42× | Current length: 1,533,033


BPE Training Progress:  82%|███████████████████████████████▏      | 200/244 [02:58<00:27,  1.60it/s]

📦 Merges: 200/244 | Current compression: 4.04× | Current length: 1,296,602


BPE Training Progress: 100%|██████████████████████████████████████| 244/244 [03:28<00:00,  1.17it/s]


📦 Merges: 244/244 | Current compression: 4.25× | Current length: 1,234,765

✅ Training complete!
✅ Final vocabulary size: 500
✅ Final compression ratio: 4.25×

📁 Saved -> /content/drive/MyDrive/bengali_tokenizer/merges.txt
📁 Saved -> /content/drive/MyDrive/bengali_tokenizer/vocab.json
📁 Saved -> /content/drive/MyDrive/bengali_tokenizer/config.json
✅ All tokenizer artifacts saved in: /content/drive/MyDrive/bengali_tokenizer


In [4]:
# --- 🔍 Test: Verify Bengali Encode/Decode Roundtrip ---
import os
import json

save_dir = '/content/drive/MyDrive/bengali_tokenizer'
merges_path = os.path.join(save_dir, 'merges.txt')
vocab_path = os.path.join(save_dir, 'vocab.json')

# --- Load merges and vocab ---
with open(merges_path, 'r', encoding='utf-8') as f:
    merges = [tuple(map(int, line.strip().split())) for line in f if line.strip()]

with open(vocab_path, 'r', encoding='utf-8') as f:
    vocab = json.load(f)

# --- Build reverse merge map ---
reverse_merges = {256 + i: pair for i, pair in enumerate(merges)}

# --- Merge function (same as training) ---
def merge(ids, pair, idx):
    newids = []
    i = 0
    while i < len(ids):
        if i < len(ids) - 1 and ids[i] == pair[0] and ids[i + 1] == pair[1]:
            newids.append(idx)
            i += 2
        else:
            newids.append(ids[i])
            i += 1
    return newids

# --- Encode ---
def encode_text(s, merges):
    ids = list(s.encode("utf-8"))
    for idx, pair in enumerate(merges, start=256):
        ids = merge(ids, pair, idx)
    return ids

# --- Recursive expand helper ---
def expand_token(i, reverse_merges):
    if i < 256:
        return [i]
    elif i in reverse_merges:
        a, b = reverse_merges[i]
        return expand_token(a, reverse_merges) + expand_token(b, reverse_merges)
    else:
        return []

# --- Decode ---
def decode_ids(ids):
    expanded = []
    for i in ids:
        expanded.extend(expand_token(i, reverse_merges))
    expanded = [b for b in expanded if 0 <= b < 256]
    return bytes(expanded).decode("utf-8", errors="ignore")

# --- 🧪 Test with 10 Bengali examples ---
test_sentences = [
    "আপনাকে অনেক ধন্যবাদ।",                          # Thank you very much.
    "বাংলা ভাষা আমার গর্ব।",                          # Bengali language is my pride.
    "আজকের আবহাওয়া সুন্দর।",                        # Today's weather is nice.
    "আমি বই পড়তে ভালোবাসি।",                       # I love reading books.
    "তুমি কোথায় যাচ্ছো?",                           # Where are you going?
    "এটা একটি ছোট গল্প।",                            # This is a short story.
    "সে খুব পরিশ্রমী ছাত্র।",                        # He is a hardworking student.
    "আমাদের স্কুল শহরে অবস্থিত।",                    # Our school is located in the city.
    "খাবারটা দারুণ লেগেছে!",                         # The food was delicious!
    "আগামীকাল সকালে সূর্যোদয় হবে।"                 # The sunrise will happen tomorrow morning.
]

print("\n🔍 Bengali Tokenizer Encode/Decode Roundtrip Test\n")

for i, sentence in enumerate(test_sentences, start=1):
    encoded = encode_text(sentence, merges)
    decoded = decode_ids(encoded)
    match = (decoded == sentence)

    print(f"🧪 Example {i}:")
    print(f"📝 Original: {sentence}")
    print(f"🧩 Encoded IDs: {encoded[:30]}{' ...' if len(encoded) > 30 else ''}")
    print(f"🔁 Decoded: {decoded}")
    print(f"✅ Match: {match}")
    print("-" * 80)

# --- Summary ---
total = len(test_sentences)
passed = sum(decode_ids(encode_text(s, merges)) == s for s in test_sentences)
print(f"\n🎯 Roundtrip Accuracy: {passed}/{total} ({100*passed/total:.1f}%)\n")



🔍 Bengali Tokenizer Encode/Decode Roundtrip Test

🧪 Example 1:
📝 Original: আপনাকে অনেক ধন্যবাদ।
🧩 Encoded IDs: [256, 134, 304, 268, 339, 439, 268, 347, 258, 167, 268, 443, 344, 318]
🔁 Decoded: আপনাকে অনেক ধন্যবাদ।
✅ Match: True
--------------------------------------------------------------------------------
🧪 Example 2:
📝 Original: বাংলা ভাষা আমার গর্ব।
🧩 Encoded IDs: [278, 412, 277, 279, 173, 480, 279, 134, 285, 302, 151, 411, 318]
🔁 Decoded: বাংলা ভাষা আমার গর্ব।
✅ Match: True
--------------------------------------------------------------------------------
🧪 Example 3:
📝 Original: আজকের আবহাওয়া সুন্দর।
🧩 Encoded IDs: [256, 134, 341, 276, 280, 134, 278, 311, 265, 147, 284, 485, 273, 357, 166, 260, 318]
🔁 Decoded: আজকের আবহাওয়া সুন্দর।
✅ Match: True
--------------------------------------------------------------------------------
🧪 Example 4:
📝 Original: আমি বই পড়তে ভালোবাসি।
🧩 Encoded IDs: [256, 134, 285, 290, 172, 333, 322, 442, 274, 270, 173, 308, 286, 278, 332, 264, 318]
🔁 Decod